In [1]:
import pandas as pd

In [2]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [3]:
df_jan = pd.read_parquet('data/fhv_tripdata_2021-01.parquet')
df_feb = pd.read_parquet('data/fhv_tripdata_2021-02.parquet')

## Q1. Downloading the data

We'll use [the same NYC taxi dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page),
but instead of "Green Taxi Trip Records", we'll use "For-Hire Vehicle Trip Records".

Download the data for January and February 2021.

Note that you need "For-Hire Vehicle Trip Records", not "High Volume For-Hire Vehicle Trip Records".

Read the data for January. How many records are there?

In [4]:
len(df_jan)

1154112

## Q2. Computing duration

Now let's compute the `duration` variable. It should contain the duration of a ride in minutes. 

What's the average trip duration in January?


In [5]:
df_jan['duration'] = df_jan.dropOff_datetime - df_jan.pickup_datetime 
df_jan['duration'] = df_jan['duration'].apply(lambda x: x.total_seconds() / 60)

In [6]:
df_jan['duration'].mean()

19.1672240937939

## Data preparation

Check the distribution of the duration variable. There are some outliers. 

Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

How many records did you drop? 

In [7]:
df_jan = df_jan[(df_jan['duration']>=1) & (df_jan['duration']<=60)]
#df_feb = data_prep(df_feb)

## Q3. Missing values

The features we'll use for our model are the pickup and dropoff location IDs. 

But they have a lot of missing values there. Let's replace them with "-1".

What's the fractions of missing values for the pickup location ID? I.e. fraction of "-1"s after you filled the NAs.

In [8]:
train_col = ['PUlocationID','DOlocationID']
target_col = ['duration']

In [9]:
df_jan[train_col] = df_jan[train_col].fillna(-1)
(df_jan['PUlocationID']==-1).value_counts(normalize=True).mul(100).astype(str)+' %'

True      83.52732770722618 %
False    16.472672292773822 %
Name: PUlocationID, dtype: object

## Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model. 

* Turn the dataframe into a list of dictionaries
* Fit a dictionary vectorizer 
* Get a feature matrix from it

What's the dimensionality of this matrix? (The number of columns).

In [10]:
df_jan[train_col]= df_jan[train_col].astype(str)
dict_values = df_jan[train_col].to_dict(orient='records')
dv = DictVectorizer()

X_train = dv.fit_transform(dict_values)
X_train.shape

(1109826, 525)

## Q5. Training a model

Now let's use the feature matrix from the previous step to train a model. 

* Train a plain linear regression model with default parameters 
* Calculate the RMSE of the model on the training data

What's the RMSE on train?


In [11]:
y_train = df_jan[target_col].values
model = LinearRegression()
model.fit(X_train,y_train)

y_pred = model.predict(X_train)

mean_squared_error(y_train,y_pred,squared=False)

10.528519107213159

## Q6. Evaluating the model

Now let's apply this model to the validation dataset (Feb 2021). 

What's the RMSE on validation?

In [19]:
def data_prep_w(df):
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    
    df = df[(df['duration']>=1) & (df['duration']<=60)]
    df[train_col] = df[train_col].apply(lambda x: x.fillna(-1))
    train_col = ['PUlocationID','DOlocationID']
    df[train_col] = df[train_col].astype(str)
    return df

def one_hot(df,enc):
    train_col = ['PUlocationID','DOlocationID']
    dict_values = df[train_col].to_dict(orient='records')
    X = enc.transform(dict_values)
    #print(X.shape)
    return X

df_feb = pd.read_parquet('data/fhv_tripdata_2021-02.parquet')
df_feb = data_prep(df_feb)


/tmp/ipykernel_9338/391501669.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[train_col] = df[train_col].astype(str)


In [21]:
y_val = df_feb[target_col].values
X_val = dv.transform(dict_values)

y_pred = model.predict(X_val)
mean_squared_error(y_val,y_pred,squared=False)


11.01428321291995

In [102]:
df_

        dispatching_base_num     pickup_datetime    dropOff_datetime  \
1            B00021          2021-02-01 00:55:40 2021-02-01 01:06:20   
2            B00021          2021-02-01 00:14:03 2021-02-01 00:28:37   
3            B00021          2021-02-01 00:27:48 2021-02-01 00:35:45   
4                     B00037 2021-02-01 00:12:50 2021-02-01 00:26:38   
5                     B00037 2021-02-01 00:00:37 2021-02-01 00:09:35   
...                      ...                 ...                 ...   
1037687               B03282 2021-02-28 23:01:16 2021-02-28 23:14:48   
1037688               B03282 2021-02-28 23:36:10 2021-02-28 23:47:38   
1037689      B03285          2021-02-28 23:18:36 2021-02-28 23:43:59   
1037690      B03285          2021-02-28 23:26:34 2021-02-28 23:44:37   
1037691      B03285          2021-02-28 23:24:00 2021-02-28 23:40:00   

        PUlocationID DOlocationID SR_Flag Affiliated_base_number   duration  
1              173.0         82.0    None        B00021  

In [106]:
def data_prep(df):
    df['duration'] = df.dropOff_datetime - df.pickup_datetime 
    df['duration'] = df['duration'].apply(lambda x: x.total_seconds() / 60)
    df = df[(df['duration']>=1) & (df['duration']<=60)]
    
    train_co = ['PUlocationID','DOlocationID']
    df.loc[:,train_co] = df[train_co].fillna(-1)
    return df

def one_hot(df,enc,train=True):
    df[train_col] = df[train_col].astype(str)
    dict_values = df[train_col].to_dict(orient='records')
    if train:
        X = enc.fit_transform(dict_values)
    else:
        X = enc.transform(dict_values)
    return X


In [142]:
df_feb

In [138]:
def data_prep(df):
    df['duration'] = df.dropOff_datetime - df.pickup_datetime 
    df['duration'] = df['duration'].apply(lambda x: x.total_seconds() / 60)
    df = df[(df['duration']>=1) & (df['duration']<=60)]
    
    train_co = ['PUlocationID','DOlocationID']
    df.loc[:,train_co] = df[train_co].fillna(-1)
    return df
print(data_prep(df_feb))

None
        dispatching_base_num     pickup_datetime    dropOff_datetime  \
1            B00021          2021-02-01 00:55:40 2021-02-01 01:06:20   
2            B00021          2021-02-01 00:14:03 2021-02-01 00:28:37   
3            B00021          2021-02-01 00:27:48 2021-02-01 00:35:45   
4                     B00037 2021-02-01 00:12:50 2021-02-01 00:26:38   
5                     B00037 2021-02-01 00:00:37 2021-02-01 00:09:35   
...                      ...                 ...                 ...   
1037687               B03282 2021-02-28 23:01:16 2021-02-28 23:14:48   
1037688               B03282 2021-02-28 23:36:10 2021-02-28 23:47:38   
1037689      B03285          2021-02-28 23:18:36 2021-02-28 23:43:59   
1037690      B03285          2021-02-28 23:26:34 2021-02-28 23:44:37   
1037691      B03285          2021-02-28 23:24:00 2021-02-28 23:40:00   

         PUlocationID  DOlocationID SR_Flag Affiliated_base_number   duration  
1                  -1            -1    None       

/tmp/ipykernel_2213/2386982711.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,train_co] = -1


In [126]:
df_feb.loc[:,train_col].fillna(-1)


,PUlocationID,DOlocationID
0,-1.0,-1.0
1,173.0,82.0
2,173.0,56.0
3,82.0,129.0
4,-1.0,225.0
...,...,...
1037687,-1.0,31.0
1037688,-1.0,169.0
1037689,28.0,171.0
1037690,16.0,252.0
